In [79]:
%load_ext autoreload
%reload_ext autoreload
import numpy as np
import matplotlib.pyplot as plt
import utils
from tqdm import tqdm
import random


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
def distance_between_all_strings(pattern, texts):
    k = len(pattern)
    motifs = []
    total_minimum_distance = 0
    for text in texts:
        minimum_motif = ""
        minimum_distance = 1000000000
        for i in range(len(text) - k):
            current_motif = text[i:i + k]
            
            current_distance = utils.hamming_distance(current_motif, pattern)
            if current_distance < minimum_distance:
                minimum_distance = current_distance
                minimum_motif = current_motif
        motifs.append(minimum_motif)
        total_minimum_distance += minimum_distance
    return total_minimum_distance, motifs


def generate_all_kmers(k):
    ans = []
    def recurse(s):
        if len(s) < k:
            recurse(s + "A")
            recurse(s + "T")
            recurse(s + "C")
            recurse(s + "G")
        else:
            ans.append(s)
    recurse("")
    return ans

def brute_find_motif(k, texts):
    kmers = generate_all_kmers(k)
    minDist = 100000000
    for kmer in kmers:
        d, _ = distance_between_all_strings(kmer, texts)
        if d < minDist:
            minDist = d
            sollution = kmer
    return sollution, minDist

In [80]:
def profile(motifs, pseudocounts=True):
    d = {"A" : 0, "T" : 1, "C" : 2, "G" : 3}
    profile = np.zeros((4,len(motifs[0])))
    for motif in motifs:
        for i, letter in enumerate(motif):
            profile[d[letter]][i] += 1
    if pseudocounts:
        return (profile + 1)/(len(motifs) + 4)
    else:
        return (profile)/len(motifs)

def most_probable_motifs(dnas, profile):
    d = {"A" : 0, "T" : 1, "C" : 2, "G" : 3}
    k = len(profile[0])
    answer_motifs = []
    for dna in dnas:
        best_motif = ""
        best_probability = 0
        for i in range(len(dnas[0])-k): # potential issue if dnas of different sizes
            current_motif = dna[i:i+k]
            current_probability = 1
            for j, letter in enumerate(current_motif):
                current_probability *= profile[d[letter]][j]
            if current_probability > best_probability:
                best_motif = current_motif
                best_probability = current_probability
        answer_motifs.append(best_motif)
    return answer_motifs

def entropy(probability_distribution):
    total = 0
    for prob in probability_distribution:
        total += prob*np.log2(prob)
    return -1 * total

def entropy_score(profile):
    k = len(profile[0])
    total_entropy = 0
    for i in range(k):
        total_entropy += entropy(profile.T[i])
    return total_entropy

def count_score(motifs):
    d = {"A" : 0, "T" : 1, "C" : 2, "G" : 3}
    prof = np.zeros((4,len(motifs[0])))
    for motif in motifs:
        for i, letter in enumerate(motif):
            prof[d[letter]][i] += 1
    prof = prof.T
    n = len(motifs)
    score = 0
    for i in range(len(prof)):
        score += n - max(prof[i])
    return score

def randomized_motif_search(dnas, k, t):
    best_motifs = []
    for i in range(t):
        start = random.randint(0, len(dnas[i])-k)
        best_motifs.append(dnas[i][start:start+k])
    while True:
        current_profile = profile(best_motifs)
        current_motifs = most_probable_motifs(dnas, current_profile)
        if count_score(current_motifs) < count_score(best_motifs):
            best_motifs = current_motifs
        else:
            return best_motifs

def iterated_randomized_motif_search(dnas, k, t, iterations):
    best_motifs = randomized_motif_search(dnas, k, t)
    for i in tqdm(range(iterations)):
        candidate_motifs = randomized_motif_search(dnas, k, t)
        if count_score(candidate_motifs) < count_score(best_motifs):
            best_motifs = candidate_motifs
    return best_motifs

['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']


In [ ]:
with open("inpt.txt") as f:
    lines = f.readlines()
    k = int(lines[0].split()[0])
    t = int(lines[0].split()[1])
    texts = list(x.strip() for x in lines[1:])
    ans = iterated_randomized_motif_search(texts, k, t, 1000)


100%|██████████| 1000/1000 [01:43<00:00,  9.62it/s]


TypeError: write() argument must be str, not list

In [85]:
with open("outpt.txt", "w") as out:
    for a in ans:
        out.write(a + "\n")